# BYO Spam Classifier (pp. 103)
## 1. Split datasets into test and training sets

In [6]:
import os
# 
os.listdir()
# os.chdir("datasets")

['easy_ham_2',
 'hard_ham',
 'easy_ham',
 'spam',
 'all_spam',
 'housing.tgz',
 'all_ham',
 'housing.csv',
 'spam_2']

In [7]:
import mailbox as mb
from mailbox import Message


ham = []
spam = []

for fn in os.listdir('easy_ham'):
    with open('easy_ham/' + fn,'rb') as file:
        ham.append(Message(file.read()))

for fn in os.listdir('spam'):
    with open('spam/' + fn,'rb') as file:
        spam.append(Message(file.read()))


In [8]:
from pprint import pprint
pprint(ham[0].items())


[('Return-Path', '<rpm-zzzlist-admin@freshrpms.net>'),
 ('Delivered-To', 'yyyy@localhost.spamassassin.taint.org'),
 ('Received',
  'from localhost (jalapeno [127.0.0.1])\n'
  '\tby jmason.org (Postfix) with ESMTP id 4B68E16F03\n'
  '\tfor <jm@localhost>; Fri,  4 Oct 2002 10:58:10 +0100 (IST)'),
 ('Received',
  'from jalapeno [127.0.0.1]\n'
  '\tby localhost with IMAP (fetchmail-5.9.0)\n'
  '\tfor jm@localhost (single-drop); Fri, 04 Oct 2002 10:58:10 +0100 (IST)'),
 ('Received',
  'from egwn.net (auth02.nl.egwn.net [193.172.5.4]) by\n'
  '    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g93M3PK16864 for\n'
  '    <jm-rpm@jmason.org>; Thu, 3 Oct 2002 23:03:26 +0100'),
 ('Received',
  'from auth02.nl.egwn.net (localhost [127.0.0.1]) by egwn.net\n'
  '    (8.11.6/8.11.6/EGWN) with ESMTP id g93Lw1f32639; Thu, 3 Oct 2002 '
  '23:58:01\n'
  '    +0200'),
 ('Received',
  'from drone5.qsi.net.nz (drone5-svc-skyt.qsi.net.nz\n'
  '    [202.89.128.5]) by egwn.net (8.11.6/8.11.6/EGWN) with SMT

In [9]:
train_ham, test_ham = ham[:-1000], ham[-1000:]
train_spam, test_spam = spam[:-200], spam[-200:]

## 2. Write a data prep pipeline to convert emails into feature vectors


In [10]:
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from functools import reduce

train_ham[0]

In [19]:
print(type(train_ham[0]))
print(type(type(train_ham[0])))

t = Type(1)
print(t)

<class 'mailbox.Message'>
<class 'type'>


NameError: name 'Type' is not defined

## 3. Add hyperparameters to prep pipeline

## 4. Try using different classifiers 